In [19]:
import pandas as pd

In [20]:
df = pd.read_csv("../Data/Processed/crimes_clear.csv")

In [21]:
# Changing the field "fecha" into a datetime type
df['fecha'] = pd.to_datetime(df['fecha'])

# Adding a new column to know the day of the week that was reporte the crime
df['dia'] = df['fecha'].dt.day_name()

# Changing names to spanish since the whole database is in spanish
day_name_mapping = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df['dia'] = df['dia'].map(day_name_mapping)

In [25]:
df.to_csv("../Data/Processed/crimes_enriched.csv", index=False)